# Setup

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pickle
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Deploy do modelo

In [ ]:
modeloTransformersImportado = ('/content/drive/MyDrive/Modulo_6_grupo_1/modelo_rede_neural_Transformers.pkl')

In [ ]:
with open(modeloTransformersImportado, 'rb') as f:
    modeloTransformers = pickle.load(f)

In [ ]:
vetorizador = SentenceTransformer('distiluse-base-multilingual-cased')

# Teste do modelo importado

In [ ]:
dadosteste = {
    'frase': ["esse o pior banco que ja vi em toda minha vida","jogou que nem o gabigol"],
}
dfteste = pd.DataFrame(dadosteste)
dfteste

,frase
0,esse o pior banco que ja vi em toda minha vida
1,jogou que nem o gabigol


In [ ]:
def prever1(dados,modelo):
  dados = vetorizador.encode(dados.tolist())
  dados = pd.DataFrame(dados)
  previsoes = modelo.predict(dados)
  previsoes = np.argmax(previsoes, axis=1)
  return previsoes

In [ ]:
prever1(dfteste["frase"],modeloTransformers)

1/1 [==============================] - 0s 277ms/step


array([2, 2])